In [1]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt
import pandas as pd

In [2]:
#Move indicator into its own Class so that will show up in output file. 
#Reference: https://www.backtrader.com/docu/inddev.html
class RSI_Oversold(bt.Indicator):
    lines = ('oversold',)
    
    params = (
        ('level', 30),    
    )

    def __init__(self):
        self.lines.oversold = bt.indicators.RSI(self.data) < self.p.level

In [3]:
class RSI_Overbought(bt.Indicator):
    lines = ('overbought',)
    
    params = (
        ('level', 70),    
    )

    def __init__(self):
        self.lines.oversold = bt.indicators.RSI(self.data) > self.p.level

In [94]:
# Create a Stratey to trade hv
class HVStrategy(bt.Strategy):
    params = (
        ("symbols", None),
        ("max_days_in_trade", 7),
        ("y", None)
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        symbols = self.p.symbols
      
        # Keep a reference to the close price, historical vol as well as returns
        self.stock_px = self.datas[symbols.index("stockpx")].close
        self.vol = self.datas[symbols.index(y)].close
        self.stock_returns = self.stock_px / self.stock_px(-1) - 1
        
        # Add indicators
#         self.sma_short = bt.indicators.SimpleMovingAverage(self.stock_px, period=50)
#         self.sma_long = bt.indicators.SimpleMovingAverage(self.stock_px, period=200)
#         self.dsma = self.sma_short - self.sma_long
#         self.mom = bt.Cmp(self.sma_short, self.sma_long)
        
        # Use this instead of below to be able to see full indicator (not just RSI) in output file
#         self.rsi_oversold = RSI_Oversold(self.stock_px)
#         self.rsi_overbought = RSI_Overbought(self.stock_px)
        self.rsi = bt.indicators.RSI(self.stock_px)
        self.rsi_oversold = bt.indicators.RSI(self.stock_px) < 30
        self.rsi_overbought = bt.indicators.RSI(self.stock_px) > 70

#         self.cross_up = bt.indicators.CrossUp(self.sma_short, self.sma_long) 
#         self.cross_down = bt.indicators.CrossDown(self.sma_short, self.sma_long)

        self.in_trade = False
        self.days_in_trade = 0
        self.flag = 0   # -1 sell, 0 no pos, 1 buy.
        
    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f; Vol, %.2f' % (self.stock_px[0], self.vol[0]))
        #==================================
        size = self.position.size

        # If not in the trade
        if (self.flag == 0):
            if (self.rsi_oversold[0] > 0 and self.days_in_trade < self.p.max_days_in_trade):
                print("Oversold indicator triggered...entering position")
                self.buy(exectype=bt.Order.Close)
                self.days_in_trade = 1
                self.flag = 1
            elif (self.rsi_overbought[0] > 0 and self.days_in_trade < self.p.max_days_in_trade):
                print("Overbought indicator triggered...entering position")
                self.sell(exectype=bt.Order.Close)
                self.days_in_trade = 1
                self.flag = -1
        
        # If in the long position
        elif (self.flag == 1):
            # Close the position if the SELL signal triggered or reached the max days to hold
            if (self.rsi_overbought[0] > 0 or self.days_in_trade == self.p.max_days_in_trade):
                self.close(exectype=bt.Order.Close)
                self.days_in_trade = 0
                self.flag = 0
                    
            elif (self.rsi_oversold[0] > 0 or self.days_in_trade < self.p.max_days_in_trade):
                self.days_in_trade += 1
                print("In position %d days" % self.days_in_trade)
        
        # If in the short position
        elif (self.flag == -1):
            # Close the position if the BUY signal triggered or reached the max days to hold
            if (self.rsi_oversold[0] > 0 or self.days_in_trade == self.p.max_days_in_trade):
                self.close(exectype=bt.Order.Close)
                print("Closing the position")
                self.days_in_trade = 0
                self.flag = 0
                    
            elif (self.rsi_overbought[0] > 0 or self.days_in_trade < self.p.max_days_in_trade):
                self.days_in_trade += 1
                print("In position %d days" % self.days_in_trade)
                
                #=================================================
                
                
#         print("Flag = %d" % self.flag)
#         print("In position %d days" % self.days_in_trade)
            
        
        # Check if we are currently in a trade
#         if(not size):
#             #If not has our indicator triggered?
#             if self.rsi_oversold[0] > 0:
#                 print("Oversold indicator triggered...entering position")
#                 self.buy(exectype=bt.Order.Close)
#                 self.days_in_trade = 1
                
#             if self.rsi_overbought[0] > 0:
#                 print("Overbought indicator triggered...entering position")
#                 self.sell(exectype=bt.Order.Close)
#                 self.days_in_trade = 1
                               
#         else:
#             self.days_in_trade += 1
#             print("In position %d days" % self.days_in_trade)
            
#             if self.days_in_trade == self.p.max_days_in_trade:
#                 print("Max days to hold trade reached...closing position")
#                 self.close(exectype=bt.Order.Close)
#                 self.days_in_trade = 0

In [95]:
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()
    
    # Add data
    datapath = os.path.join('../../../../datas/YHOO.csv')     ### HERE ###
    
    datas = pd.read_csv(datapath, parse_dates=True,index_col=0)
    
    y = '20dORHV' #iv60   ### HERE ###
    
    symbol_cols = [y,'stockpx'] #symbol we're trying to forecast/trade first
    #symbol_cols = list(datas) #Load all symbols
    
    
    # Add the Data Feeds to Cerebro
    for i, symbol in enumerate(symbol_cols):
        data = datas[[symbol]]
        data.columns = ["Close"]
        cerebro.adddata(bt.feeds.PandasData(dataname=data), name=symbol)
    
    # Add a strategy
    cerebro.addstrategy(HVStrategy, symbols=symbol_cols, y=y)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Write output
    cerebro.addwriter(bt.WriterFile, out='hv20-rsi-YHOO-MAX7.csv',csv=True)   ### HERE ###
    
    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2010-01-04, Close, 17.10; Vol, 24.31
2010-01-05, Close, 17.22; Vol, 24.37
2010-01-06, Close, 17.16; Vol, 24.14
2010-01-07, Close, 16.70; Vol, 25.13
2010-01-08, Close, 16.70; Vol, 24.18
2010-01-11, Close, 16.80; Vol, 23.75
2010-01-12, Close, 16.69; Vol, 21.77
2010-01-13, Close, 16.88; Vol, 21.43
2010-01-14, Close, 17.17; Vol, 22.15
2010-01-15, Close, 16.85; Vol, 22.37
2010-01-19, Close, 16.74; Vol, 22.00
2010-01-20, Close, 16.40; Vol, 21.83
2010-01-21, Close, 16.22; Vol, 21.48
2010-01-22, Close, 15.86; Vol, 22.20
2010-01-25, Close, 15.88; Vol, 20.78
Oversold indicator triggered...entering position
2010-01-26, Close, 15.98; Vol, 20.19
In position 2 days
2010-01-27, Close, 15.96; Vol, 22.15
In position 3 days
2010-01-28, Close, 15.48; Vol, 23.65
In position 4 days
2010-01-29, Close, 15.08; Vol, 24.68
In position 5 days
2010-02-01, Close, 15.05; Vol, 25.00
In position 6 days
2010-02-02, Close, 15.16; Vol, 24.84
In position 7 days
2010-02-03, Close, 15.46

2